In [202]:
from __future__ import annotations
import numpy as np
from scipy.signal import convolve2d
import tkinter as tk
from itertools import product
from math import floor, prod
import random
from random import choice
from numpy import ndarray


In [70]:
from pickletools import int4
from random import choices


class Game:
    def __init__(
            self,
            grid: Grid,
        ) -> None:
        self.grid = grid
        # What the player can see
        self.player_grid_view = np.zeros(grid.grid_shape(), dtype=bool)
    
    def action(self, x: int, y: int):# Discover new part (or not new) of the grid
        if np.sum(self.player_grid_view) == 0:
            # First action of the player, we move the grid such that the player input is on a empty cell
            self.grid.move_to_empty(x, y)

        self.player_grid_view = np.logical_or(
            self.player_grid_view,
            self.grid.discover(x, y),
        )
    
    def is_ended(self):
        result = self.result()
        return result is not None
    
    def result(self):
        # Discovered a mine
        if np.any(np.logical_and(self.player_grid_view, self.grid.mines)):
            return False
        
        # Discovered has many tile than free tiles
        return None if self.grid.n_bomb + np.sum(self.player_grid_view) < prod(self.grid.grid_shape()) else True
    
    def visible_grid(self):
        return self.grid.grid*(self.player_grid_view), self.player_grid_view

        


class Grid:
    def __init__(self, n_row: int, n_col: int, bomb_percent:float) -> None:
        self.grid = np.zeros((n_row, n_col))

        self.mines = np.zeros((n_row, n_col), dtype=bool)
        # Place bombs
        self.n_bomb = floor(n_row*n_col*bomb_percent)
        bomb_coordinates = random.sample(list(product(range(n_row), range(n_col))), self.n_bomb)
        for x, y in bomb_coordinates:
            self.mines[x, y] = True

        # Compute the adjacents bombs
        self.grid = convolve2d(self.mines, np.ones((3, 3)), mode='same')

    def move_to_empty(self, x: int, y: int)-> int:
        # Find a good spot to move
        values = np.copy(self.grid)
        values[self.mines] = np.max(self.grid)+1
        min_value = np.min(values)
        if values[x, y] == min_value:
            return values[x, y]

        possible_destinations = np.argwhere(values == min_value)
        destination: np.ndarray = choices(possible_destinations, k=1)[0]

        # Slide the grid
        self.slide_grid(*(np.array([x, y]) - destination))
        # On edge cases, we might still be not on the lowest possible grid value
        # We itrate until it is stable
        return self.move_to_empty(x, y)



    def slide_grid(self, delta_x: int, delta_y: int):
        self.mines = np.roll(
            np.roll(
                self.mines,
                delta_x,
                axis=0,
            ),
            delta_y,
            axis=1,
        )

        self.update()


    def update(self):
        # Update the number of adjacents if the mines array have been changed
        self.grid = convolve2d(self.mines, np.ones((3, 3)), mode='same')

    def grid_shape(self):
        return self.mines.shape
    
    def discover(self, x:int, y: int):
        if self.grid[x, y] > 0: # Cover the case of a mine
            result = np.zeros_like(self.mines)
            result[x, y] = True
            return result
        
        # Expand the area
        return self.expand(x, y)

    def expand(self, x:int, y:int):
        result = np.zeros_like(self.mines)
        result[x, y] = True
        last_result = np.zeros_like(self.mines)
        while np.any(last_result != result): # Check if stabilized
            last_result = result
            
            # Restrict
            result = np.logical_and(
                result,
                self.grid == 0,
            )
            
            # Expand
            result = convolve2d(result, np.ones((3, 3)), mode='same').astype(bool)

            
        return result
        
class Player_Interface:
    def action(self, grid: np.ndarray, grid_view: np.ndarray) -> tuple[int, int]:
        pass

class Random_Player(Player_Interface):
    def action(self, grid: np.ndarray, grid_view: np.ndarray):
        return random.randint(0, grid.shape[0]-1), random.randint(0, grid.shape[1]-1)
    

class Command_Line_UI:
    def start(self, game: Game, player:Player_Interface):
        print('Game start')
        print(f'Grid size : {game.grid.grid_shape()}')
        while not game.is_ended(): # Player has not discovered all cases
            next_action = player.action(*game.visible_grid())
            print(f'Player plays {next_action}')
            
            game.action(*next_action)
        
        result = game.result()
        if result:
            print('Player win')
        else:
            print('Player lose')
        return result

In [71]:
import tkinter as tk
from tkinter import messagebox
import random

class GUI_User_Inputs:
    def __init__(self, master: tk.Tk | None=None):
        self.master = master if master is not None else tk.Tk()
        self.master.title("Minesweeper")
        self.master.geometry("1440x720")

        # Create the structure to place the game grid
        self.create_widgets()

    def start(self, game: Game):
        # Update grid of the minesweeper
        self.initialize_grid(game)

        # Lunch the app
        self.master.mainloop()

        return game.result()        

    def create_widgets(self):
        self.grid_frame = tk.Frame(self.master)
        self.grid_frame.pack(pady=10)

        self.status_bar = tk.Label(self.master, text=f"Total number of mines : Unknow", bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X)

        # self.initialize_grid()
        # self.generate_mines(10)

    def initialize_grid(self, game: Game):
        # Initialize flag storage
        self.flags = np.zeros_like(game.player_grid_view)

        # Create the buttons/grid
        self.buttons: list[list[tk.Button]] = []
        for row in range(game.grid.grid_shape()[0]):
            row_buttons = []
            for col in range(game.grid.grid_shape()[1]):
                button = tk.Button(self.grid_frame, width=2, height=1, command=lambda r=row, c=col: self.on_button_click(game, r, c))
                button.grid(row=row, column=col)
                button.bind("<Button-3>", lambda e, r=row, c=col: self.on_right_click(game, r, c))
                row_buttons.append(button)
            self.buttons.append(row_buttons)

        # Upadte the label
        self.status_bar.config(text=f"Total number of mines : {game.grid.n_bomb}")

        self.update_grid(game)

    def on_button_click(self, game: Game, row: int, col: int):
        game.action(row, col)

        self.update_grid(game)

    def on_right_click(self, game: Game, row: int, col: int):
        self.flags[row, col] = not self.flags[row, col]
        self.update_grid(game)

    def update_grid(self, game: Game):
        # Place the numbers on the buttons
        for row in range(game.grid.grid_shape()[0]):
            for col in range(game.grid.grid_shape()[1]):
                button = self.buttons[row][col]

                if not game.player_grid_view[row, col]:
                    if self.flags[row, col]:
                        text_button = 'F'
                        color='yellow'
                    else:
                        text_button = ''
                        color='gray'
                elif game.grid.mines[row, col]:
                    text_button = 'M'
                    color='red'
                else:
                    value = int(game.grid.grid[row, col])
                    text_button = str(value) if value != 0 else ''
                    color='lightgrey'

                button.config(text=text_button, bg=color)

# gui = GUI_User_Inputs()
# grid = Grid(30, 30, 0.1)
# game = Game(grid)

# game_result = gui.start(game)
# game_result

In [249]:
class GUI_Bot_Inputs(GUI_User_Inputs):
    def __init__(self, master: tk.Tk | None = None, delay:int=1000):
        super().__init__(master)
        self.master.bind("<FocusIn>", self.on_focus_in)
        self.delay = delay

    def on_focus_in(self, event):
        self.bot_play()

    def bot_play(self):
        if not self.game.is_ended():
            next_action = self.player.action(*self.game.visible_grid())
            self.on_button_click(self.game, *next_action)

            # Add flags
            try:
                flags = np.argwhere(self.player.known_mines)
                for x, y in flags:
                    if not self.flags[x, y]:
                        self.on_right_click(self.game, x, y)
            except:
                pass
            self.master.after(self.delay, self.bot_play)



    def start(self, game: Game, player: Player_Interface):
        self.game = game
        self.player = player
        return super().start(game)

In [240]:
class Minesweeper_bot(Player_Interface):
    def action(self, grid: ndarray, grid_view: ndarray) -> tuple[int, int]:
        self.grid = grid
        self.grid_view = grid_view

        # Only make sense on the ~grid_view array
        self.unkown_region = ~grid_view
        self.known_mines = np.zeros_like(grid_view)
        # self.known_no_mines = np.copy(grid_view) # Useless, if we found a known_no_mines, we return it imediatly

        self.to_inspect = list(np.argwhere(np.logical_and(
            grid_view,
            grid > 0
        )))
        for x, y in self.to_inspect:
            solution = self.inspect(x, y)
            if solution is not None:
                return solution
        
        # No action have been deduce, we sample a possible move where we do not know if there is a bomb
        possible_actions = np.argwhere(np.logical_and(
            ~grid_view,
            ~self.known_mines,
        ))

        return choice(possible_actions)


    def all_neighbors(self, x: int, y: int):
        # List all neigbhors
        neighbors = [
            (x+dx, y+dy) 
            for dx in (-1, 0, 1) for dy in (-1, 0, 1) 
            if 0 <= x+dx and x+dx < self.known_mines.shape[0]
            and 0 <= y+dy and y+dy < self.known_mines.shape[1]
            and not (dx==0 and dy==0)
        ]
        return neighbors

    def inspect(self, x: int, y: int) -> tuple[int, int] | None:
        # List all neigbhors
        neighbors = self.all_neighbors(x, y)
        value = self.grid[x, y]

        # Case value == 0 isn't interestng because all the neighbors are values
        if self.grid_view[x, y] and value > 0:
            unknown_boxes = [
                (i, j)
                for i, j in neighbors
                if self.unkown_region[i, j]
            ]
            if len(unknown_boxes) == value:
                # All bombs
                for i, j in neighbors:
                    if self.unkown_region[i, j] and not self.known_mines[i, j]:
                        self.known_mines[i, j] = True
                        # Might give information to neighbors of the mine
                        self.to_inspect += [n for n in self.all_neighbors(i, j) if self.grid_view[*n]]
            else:
                # Check if all the mines have been founded
                founded_mines = [
                    (i, j)
                    for i, j in neighbors
                    if self.known_mines[i, j]
                ]
                if len(founded_mines) == value:
                    # All remaining boxes are safe
                    for i, j in neighbors:
                        if self.unkown_region[i, j] and not self.known_mines[i, j]:
                            return i, j
                else:
                    # No decision can be made
                    pass

        # We didn't deduce
        return None


In [266]:
gui_bot = GUI_Bot_Inputs(delay=100)
grid = Grid(20, 20, 0.2)
game = Game(grid)
bot = Minesweeper_bot()

game_result = gui_bot.start(game, bot)
game_result

False

In [225]:
grid.grid

array([[1., 2., 1., 2., 1., 1., 0., 0., 0., 0.],
       [2., 3., 2., 2., 1., 1., 0., 0., 0., 0.],
       [2., 3., 2., 1., 0., 1., 1., 1., 1., 1.],
       [1., 3., 4., 4., 3., 3., 2., 1., 1., 1.],
       [0., 2., 3., 4., 3., 4., 3., 2., 2., 2.],
       [0., 1., 3., 5., 6., 5., 3., 2., 2., 2.],
       [0., 1., 2., 3., 3., 3., 2., 2., 2., 2.],
       [0., 1., 2., 3., 3., 2., 1., 1., 1., 1.],
       [0., 1., 1., 1., 0., 0., 0., 1., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 1., 2., 2.]])

In [222]:
game = Game(grid)
game_result = Command_Line_UI().start(game, bot)
game_result

Game start
Grid size : (10, 10)
Player plays [1 6]
Player plays [9 6]
Player plays (6, 6)
Player plays (6, 7)
Player plays (6, 2)
Player plays (5, 2)
Player plays (5, 3)
Player plays [0 1]
Player plays [3 8]
Player plays (3, 7)
Player plays (4, 6)
Player plays (4, 7)
Player plays (4, 8)
Player plays (4, 9)
Player plays (1, 4)
Player plays (0, 3)
Player plays (1, 3)
Player plays (2, 3)
Player plays (2, 4)
Player plays (2, 2)
Player plays (1, 2)
Player plays (5, 4)
Player plays (5, 5)
Player plays (5, 7)
Player plays (5, 8)
Player plays (6, 9)
Player plays (7, 8)
Player plays (7, 9)
Player plays (8, 8)
Player plays (8, 9)
Player plays [1 0]
Player lose


False